# <center> Predicting Donors Choose</center>

--------

# Introduction
*This notebook will use features designed more for NN:*
- one hot encoding
- seperate enviroment

## Imports

https://lightgbm.readthedocs.io/en/latest/Installation-Guide.html

In [160]:
import gc
import numpy as np
import pandas as pd
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm_notebook

In [161]:
#import lightgbm as lgb

In [162]:
from sklearn.preprocessing import StandardScaler

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import re

In [163]:
#I would rather not run the while dataset on my laptop
# so kaggle determines if I am also evaluating the kaggle test set
kaggle = False

#sim refers to if data simulating variable situations is being generated & tested on. 
sim = False

#Kaggle & Sim use 'test' so dont use both

In [164]:
train = pd.read_csv('./Input/train.csv', low_memory=False, index_col='id')
if kaggle: 
    if sim == False:
        test = pd.read_csv('./Input/test.csv', low_memory=False, index_col='id')

res = pd.read_csv('./Input/resources.csv', low_memory=False, index_col='id')
train['project_submitted_datetime'] = pd.to_datetime(train['project_submitted_datetime'])

In [165]:
train = train[:10000]

### Simulated Data

In [166]:
def make_sim(n='p039565'):
    """
    Makes a semi-brute forced dataset based off the entry n, with variable:
    Months (jan-dec), 
    Pronouns (not teacher or dr), 
    Previous Entries(0:30),
    to see what combination may have a higher success rate
    """
    test = pd.DataFrame(train.loc[n]).transpose()
    del test['project_is_approved']
    
    change_dict = {"prefix":['Mrs.','Ms.','Mr.'],
        "date":['2017-01-26','2017-02-26','2017-03-26','2017-04-26','2017-05-26','2017-06-26',
        '2017-07-26','2017-08-26','2017-09-26','2017-10-26','2017-11-26','2017-12-26',],
        "prev" : [x for x in range(0,10)]}

    for i in range(100):
        test.loc[n+str(i)] = test.loc[n].transpose()
        test.set_value(n+str(i),'teacher_prefix',np.random.choice(change_dict['prefix']))
        test.set_value(n+str(i),'project_submitted_datetime',np.random.choice(change_dict['date']))
        test.set_value(n+str(i),'teacher_number_of_previously_posted_projects',np.random.choice(change_dict['prev']))
        
    test.index.rename = 'id'
    return(test)

In [167]:
#making a dataset which has variable situations that 'p039654' may be under
if sim:
    test = make_sim('p039565')
    train = train.drop('p039565',axis=0)
    del make_sim

## Preprocessing

### 1.1 Resource Intergration
Here we evaluate how much each project/proposal will cost and/or how big they are

In [168]:
res['cost'] = res['quantity'] * res['price']
res_agg = res.groupby('id').agg({'description': ['nunique'], 'quantity': ['sum'], 'cost': ['mean', 'sum']})
res_agg.columns = ['unique_items', 'total_quantity', 'mean_cost', 'total_cost']
res_agg.reset_index(inplace=True)

#description was dropped because the description of the project should not have an effect on its liklihood of success

In [169]:
train = train.merge(res_agg, left_index=True, right_on='id')
if kaggle: 
    if sim : 
        for i in res_agg.columns:
            if i != 'id':
                test[i] = res_agg[res_agg['id']=='p039565'].drop('id',axis=1)[i].values[0]
    else:
        test =  test.merge(res_agg, left_index=True, right_on='id')

del res_agg
del res

### 1.2 Preprocessing of features

#### EDA

In [170]:
"""
train_resource["project_submitted_datetime"] = pd.to_datetime(train_resource["project_submitted_datetime"])
train_resource["month_created"] = train_resource["project_submitted_datetime"].dt.month
loan = train_resource.groupby(['school_state', 'month_created'])['price'].mean().unstack()
loan = loan.sort_values([3], ascending=False)
f, ax = plt.subplots(figsize=(15, 20)) 
loan = loan.fillna(0)
temp = sns.heatmap(loan, cmap='Reds')
plt.show()
"""

'\ntrain_resource["project_submitted_datetime"] = pd.to_datetime(train_resource["project_submitted_datetime"])\ntrain_resource["month_created"] = train_resource["project_submitted_datetime"].dt.month\nloan = train_resource.groupby([\'school_state\', \'month_created\'])[\'price\'].mean().unstack()\nloan = loan.sort_values([3], ascending=False)\nf, ax = plt.subplots(figsize=(15, 20)) \nloan = loan.fillna(0)\ntemp = sns.heatmap(loan, cmap=\'Reds\')\nplt.show()\n'

In [171]:
#gen_mon = train.groupby(['datetime_month','gender'])['project_is_approved'].mean().reset_index()
#prev_mon = train.groupby('teacher_number_of_previously_posted_projects')['project_is_approved'].mean().reset_index()

#bins = pd.cut(train['teacher_number_of_previously_posted_projects'], [0, 25, 50, 75, 100, 125, 150])
#pros = train.groupby(bins)['project_is_approved'].agg('mean')

In [172]:
#sns.barplot(pros.index,pros)
#plt.title('Frequent Applicants')
#plt.xlabel("Teacher's Previous Postings")
#plt.ylabel("Mean Approvals")

In [173]:
#train.project_is_approved.value_counts()

In [174]:
#sns.lmplot(x="datetime_month", y="project_is_approved", hue="gender", 
#           data=gen_mon[gen_mon['gender']!='Unk'], 
#           order=3,ci=80)
#plt.title('Approval rate, by month, by gender (ci:90)')

In [175]:
#del gen_mon
#del prev_mon

In [176]:
gc.collect()

409

#### Cat Preprocessing

There are 51 States, because of 50 + DC

In [177]:
train.isnull().sum()[train.isnull().sum()>0]

project_essay_3    9648
project_essay_4    9648
dtype: int64

In [178]:
train.teacher_prefix[train.teacher_prefix.isnull()] = 'Teacher'
try: 
    test.teacher_prefix[test.teacher_prefix.isnull()] = 'Teacher'
except:
    pass

/Users/Nahel/anaconda3/envs/NN/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


fill na did not work

In [179]:
def date_prep(train):
    train['datetime_year'] = train['project_submitted_datetime'].dt.year
    train['datetime_month'] = train['project_submitted_datetime'].dt.month
    return(train)

In [180]:
def gender_features(train):
    conditions = [(train.teacher_prefix == 'Mr.'), 
                  (train.teacher_prefix == 'Mrs.') | (train.teacher_prefix == 'Ms.')]
    choices = ['Male', 'Female']
    train['gender'] = np.select(conditions, choices, default='Unk')
    return(train)

In [181]:
date_prep(train)
if kaggle: date_prep(test)

train = gender_features(train)
if kaggle: test = gender_features(test)
    
del train['project_submitted_datetime']
del train['project_subject_subcategories']
del date_prep, gender_features

In [182]:
gc.collect()

315

##### Encoding labels

In [183]:
# Because of memory issues, it made more sense to encode everything as a string rather than dummies
cols  = ['gender',
        'teacher_prefix', 
        'school_state',
        'project_grade_category',
         'datetime_year',
         'datetime_month',]

for c in tqdm_notebook(cols):
    dummies = pd.get_dummies(train[c])
    train = train.join(dummies)
    del train[c]

HBox(children=(IntProgress(value=0, max=6), HTML(value='')))

#### Num Preprocessing

In [184]:
num_features  = ['teacher_number_of_previously_posted_projects',
                 'total_quantity', 'mean_cost', 'total_cost','unique_items']

In [185]:
SS = StandardScaler()
train[num_features] = SS.fit_transform(train[num_features])
if kaggle: test[num_features] = SS.fit_transform(test[num_features])

In [186]:
del num_features, StandardScaler, SS

In [187]:
gc.collect()

133

#### Text Preprocessing

In [188]:
other_stopwords = [x for x in 'abcdefghijklmnopqrstuvwxyz']
for j in ['student','students','education',]:
    other_stopwords.append(j)

In [189]:
#import gensim
from nltk.corpus import stopwords
def scrub(text):
    text = text.strip().lower()
    text = re.sub('\W+',' ', text)    
    text = re.sub(r'_', ' ', text)
    text = re.sub(r'\t', ' ', text)
    text = re.sub(r'\:', ' ', text)
    text = re.sub(r'\+', ' ', text)
    text = re.sub(r'\=', ' ', text)
    text = re.sub(r'(\")', ' ', text)
    text = re.sub(r'(\r)', ' ', text)
    text = re.sub(r'(\n)', ' ', text)
    text = re.sub(r'(\\)', ' ', text)
    text = re.sub('n t ','n\'t ', text)
    text = re.sub(' re ',' are ', text)
    text = re.sub(r'(\r\n)', ' ', text)
    text = re.sub(r'\"\"\"\"', ' ', text)
    text = re.sub(' i m ',' i\'m ', text)
    return(text)
def swords(text):
    text = ' '.join([word for word in text.split() if word not in (other_stopwords + stopwords.words("english"))])
    return(text)

##### Project Category Preprocessing

In [190]:
from sklearn.feature_extraction.text import CountVectorizer

In [191]:
ctv = CountVectorizer()
project_cats = train.project_subject_categories.apply(lambda x: scrub(x))
x = ctv.fit_transform(project_cats)
train = train.reset_index

In [192]:
train = train().merge(pd.DataFrame(x.toarray(), 
                         columns=["subject_cat_"+nm for nm in ctv.get_feature_names()])
            ,left_index=True,
            right_index=True)
del ctv, x ,project_cats, train['project_subject_categories']

##### Essay Preprocessing

  
#### Before May 17th, 2016:

- project_essay_1: "Introduce us to your classroom"
- project_essay_2: "Tell us more about your students"
- project_essay_3: "Describe how your students will use the materials you're requesting"
- project_essay_4: "Close by sharing why your project will make a difference"

#### May 17th, 2016 and beyond:

- project_essay_1: "Describe your students: What makes your students special? Specific details about their background, your neighborhood, and your school are all helpful."
- project_essay_2: "About your project: How will these materials make a difference in your students' learning and improve their school lives?"

#### Plan
- Combine essay_1 and essay_2 before May 17th to make "student_description" and use essay_1 after May 17th directly
- Combine essay_3 and essay_4 before May 17th to make "project_description" and use essay_2 after May 17th directly

In [193]:
def essay_convert(train):
    # Making the First essay : student_description
    train['student_description']=train['project_essay_1']
    train.loc[train.project_essay_3.notnull(),'student_description']=train.loc[train.project_essay_3.notnull(),'project_essay_1']+train.loc[train.project_essay_3.notnull(),'project_essay_2']

    # Making the second essay : project_description
    train['project_description']=train['project_essay_2']
    train.loc[train.project_essay_3.notnull(),'project_description']=train.loc[train.project_essay_3.notnull(),'project_essay_3']+train.loc[train.project_essay_3.notnull(),'project_essay_4']

    # Removing original essays
    del train['project_essay_1']
    del train['project_essay_2']
    del train['project_essay_3']
    del train['project_essay_4']
    return(train)

In [194]:
essay_convert(train)
if kaggle: essay_convert(test)
gc.collect()

del essay_convert

##### Scrub Text Features

In [195]:
text_features = ['project_title', 'project_resource_summary',
                'project_description', 'student_description']

In [196]:
for j in tqdm_notebook(text_features):
    n_col = 'processed_'+j
    train[n_col] = train[j].apply(lambda x: scrub(x)).apply(lambda x: swords(x))
    if kaggle: test[n_col] = test[j].apply(lambda x: scrub(x)).apply(lambda x: swords(x))

gc.collect()
for i in text_features:
    del train[i]
    if kaggle: del test[i]

del stopwords, other_stopwords, text_features, swords

HBox(children=(IntProgress(value=0, max=4), HTML(value='')))

In [197]:
gc.collect()

28

## Modeling

### Tfidf & X,y Assignment

In [198]:
gc.collect()

0

In [199]:
cols = [
    'processed_project_title',
    'processed_project_resource_summary', 
    'processed_project_description',
    'processed_student_description']

In [200]:
for i in cols:
    print("Average length in {} is {} words".format(i,str(round(train[i].str.len().mean()))))

Average length in processed_project_title is 25 words
Average length in processed_project_resource_summary is 80 words
Average length in processed_project_description is 524 words
Average length in processed_student_description is 413 words


In [201]:
n_features = [
    100, 
    200, 
    1500,
    1500]

In [202]:
gc.collect()

122

In [203]:
for c_i, c in tqdm_notebook(enumerate(cols)):
    tfidf = TfidfVectorizer(
        ngram_range=(1,3),
        max_features=n_features[c_i])

    tfidf.fit(train[c])

#    tfidf_train2 = tfidf.transform(train[c])

    tfidf_train = np.array(tfidf.transform(train[c].values).toarray(), dtype=np.float16)
    for i in range(n_features[c_i]):
        train[c + '_tfidf_' + str(i)] = tfidf_train[:, i]
    if kaggle:
        tfidf_test = np.array(tfidf.transform(test[c].values).toarray(), dtype=np.float16)
        for i in range(n_features[c_i]):
            test[c + '_tfidf_' + str(i)] = tfidf_test[:, i]

for i in cols:
    del train[i]
    if kaggle: del test[i]

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [204]:
#did this a while ago
#train = train.reset_index()
del train['index']

In [205]:
drop_cols = ['project_is_approved','id','teacher_id']

X = train.drop(drop_cols, axis=1)
y = train['project_is_approved']
feature_names = list(X.columns)
del train

In [206]:
#if running demo, uncomment this and change test['id'].vaues -> test['index'].values
#test = test.reset_index()

In [207]:
#X_test = test.drop(drop_cols, axis=1, errors='ignore')
#id_test = test['id'].values

### Keras Dense

In [209]:
from sklearn.model_selection import train_test_split

In [244]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [245]:
y_train.shape

(7500,)

In [246]:
X_train.shape[1]

3400

In [247]:
import keras
from keras import regularizers, optimizers

from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.callbacks import EarlyStopping, Callback

In [277]:
adam = optimizers.Adam(lr=0.01)
l2 = regularizers.l2(0.5)

In [279]:
model = Sequential()
model.add(Dense(1028,input_shape=(X_train.shape[1],), 
                activation='relu',
                kernel_regularizer = l2))
model.add(Dropout(0.25))

model.add(Dense(256, 
                activation='relu',
                kernel_regularizer = l2))

model.add(Dense(1, 
                activation='sigmoid'))

In [282]:
model.compile(loss = 'binary_crossentropy',
             optimizer = adam)

early_stop = EarlyStopping(monitor='val_loss', min_delta=1, patience=10)

callbacks_list = [early_stop]

In [283]:
from sklearn.metrics import roc_auc_score
class roc_callback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.x)
        roc = roc_auc_score(self.y, y_pred)
        y_pred_val = self.model.predict(self.x_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        print('\rroc-auc: %s - roc-auc_val: %s' % (str(round(roc,4)),str(round(roc_val,4))),end=100*' '+'\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), 
                    epochs=5,
                    callbacks=[roc_callback(training_data=(X_train, y_train),
                                            validation_data=(X_test, y_test))])

Train on 7500 samples, validate on 2500 samples
Epoch 1/5
7500/7500 [==============================] - 67s 9ms/step - loss: 20.7933 - val_loss: 0.4315
roc-auc: 0.6183 - roc-auc_val: 0.5781                                                                                                    
Epoch 2/5
7500/7500 [==============================] - 69s 9ms/step - loss: 0.4333 - val_loss: 0.4233
roc-auc: 0.6072 - roc-auc_val: 0.5815                                                                                                    
Epoch 3/5
7500/7500 [==============================] - 76s 10ms/step - loss: 0.4280 - val_loss: 0.4258
roc-auc: 0.4355 - roc-auc_val: 0.4622                                                                                                    
Epoch 4/5
7500/7500 [==============================] - 70s 9ms/step - loss: 0.4274 - val_loss: 0.4281
roc-auc: 0.6017 - roc-auc_val: 0.6129                                                                                           

## Prediction

In [ ]:
#p = model.predict(X_test, num_iteration=model.best_iteration)

In [ ]:
idf = tfidf.idf_
idf_map = dict(zip(tfidf.get_feature_names(), idf))

In [ ]:
tfidf.get_feature_names()[75]

In [ ]:
tfidf.idf_[75]

In [ ]:
#idf_map

In [ ]:
submit = pd.DataFrame({'id':id_test,'pred':p})

In [ ]:
#submit.sort_values('pred',axis=0,)